In [1]:
import pandas as pd
import lightgbm as lgb
import joblib
import os
from sklearn.model_selection import train_test_split

print("--- Entraînement du Modèle de Risque NEUTRE (pour calibration) ---")

# --- 1. Chargement et Préparation ---
input_path = "../../data/processed/dataset_clean_no_outliers.parquet"
output_dir = "../../models/"
df = pd.read_parquet(input_path)
df['is_default'] = df['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)

# --- 2. Sélection des Variables ---
selected_features = [
    'term', 'annual_inc', 'dti', 'revol_util', 'revol_bal', 'loan_amnt', 
    'emp_length', 'home_ownership', 'purpose', 'verification_status', 
    'mort_acc', 'pub_rec', 'open_acc', 'total_acc'
]
X = df[selected_features]
y = df['is_default']
X = pd.get_dummies(X, drop_first=True, dtype=float)

# --- 3. Division et Entraînement ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# LA MODIFICATION CLÉ : On retire 'scale_pos_weight' pour un modèle non biaisé
model_1_neutral = lgb.LGBMClassifier(objective='binary', metric='auc', random_state=42)

print("Entraînement du modèle neutre...")
model_1_neutral.fit(X_train, y_train)
print("✅ Modèle de risque neutre entraîné.")

# --- 4. Sauvegarde ---
# On le sauvegarde sous un nouveau nom pour ne pas écraser l'ancien
model_path = os.path.join(output_dir, "step1_risk_model_NEUTRAL.pkl")
joblib.dump(model_1_neutral, model_path)
print(f"\n✅ Modèle neutre sauvegardé dans : {model_path}")

--- Entraînement du Modèle de Risque NEUTRE (pour calibration) ---
Entraînement du modèle neutre...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 56440, number of negative: 236531
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007484 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1529
[LightGBM] [Info] Number of data points in the train set: 292971, number of used features: 31
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.192647 -> initscore=-1.432901
[LightGBM] [Info] Start training from score -1.432901
✅ Modèle de risque neutre entraîné.

✅ Modèle neutre sauvegardé dans : ../../models/step1_risk_model_NEUTRAL.pkl
